# 메타 프롬프트 생성 에이전트

> 참고: https://wikidocs.net/267817

사용자로 부터 요구사항을 수집하고, 이를 바탕으로 프롬프트를 작성합니다.

In [3]:
import os
import getpass
from dotenv import load_dotenv

load_dotenv("../.env", override=True)


def _set_env(var: str):
    env_value = os.environ.get(var)
    if not env_value:
        env_value = getpass.getpass(f"{var}: ")

    os.environ[var] = env_value


_set_env("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "02-LangGraph-Prompt-Generation"
_set_env("OPENAI_API_KEY")

## 상태 정의

In [4]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list, add_messages]

## 사용자 요구사항 수집 노드

In [ ]:
from pydantic import BaseModel
from langchain_openai import ChatOpenAI


class PromptInstructions(BaseModel):
    """프롬프트하는 방법에 대한 지침"""

    objective: Annotated[str, "프롬프트의 목표"]
    variables: Annotated[list[str], "프롬프트 템플릿에 전달될 변수 목록"]
    constraints: Annotated[list[str], "출력에서 피해야 할 제약 조건 목록"]
    requirements: Annotated[list[str], "출력이 반드시 따라야 할 요구사항 목록"]


def uses_requirements_node(state: State):
    system_prompt = """당신의 임무는 사용자로부터 어떤 유형의 프롬프트 템플릿을 생성하고자 하는지 정보를 수집하는 것입니다.

사용자로부터 다음 정보를 수집해야 합니다:

- 프롬프트의 목적
- 프롬프트 템플릿에 전달될 변수
- 출력이 해서는 안 되는 사항에 대한 제약 조건
- 출력이 반드시 준수해야 하는 요구사항

이 정보를 파악할 수 없다면 명확히 설명해 달라고 요청하세요! 무턱대고 추측하지 마십시오.

모든 정보를 파악한 후 관련 도구를 호출하십시오.

[중요] 대화는 한국어로 진행하지만, 생성된 프롬프트는 영어로 작성되어야 합니다."""

    llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0).bind_tools(
        [PromptInstructions]
    )

    response = llm.invoke([("system", system_prompt)] + state["messages"])

    return {"messages": [response]}
